The following code processes monthly faf-water SST data, using ocean potential temperature fields (thetao) or sea surface temperature (tos) ESGF and the intake-esm datastore, producing the following files used in the analysis notebooks: SST_response_maps_fafwater.nc, and fafwater_CRFs_unscaled.nc

For the response maps, all data is regridded to a common 1x1 grid. Large memory allocations are required to process the piControl data (100 GB used here). 

Directory paths must be changed to a user's local environment if reproducing 

In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import xesmf as xe
import gsw
from glob import glob
import netCDF4
import gc

from xmip.preprocessing import rename_cmip6
from xmip.preprocessing import broadcast_lonlat
from xmip.preprocessing import promote_empty_dims
from xmip.preprocessing import combined_preprocessing
from xmip.utils import google_cmip_col
from xmip.postprocessing import interpolate_grid_label
from xmip.postprocessing import pick_first_member
from xmip.postprocessing import concat_members

import dask
import dask.array as da
import dask_jobqueue
from dask.distributed import Client, config
from dask.diagnostics import ProgressBar


%matplotlib inline 
import warnings
warnings.filterwarnings("ignore")

In [2]:
########################################################################
# extract contributing model names for a given directory/experiment

def extract_model_names(directory,experiment):
    model_names = []
    file_pattern = os.path.join(directory, '*_*_{}.nc'.format(experiment))

    for file_path in glob(file_pattern):
        filename = os.path.basename(file_path)
        model_name = filename.split('_')[1]  # Extract the model name from the filename
        model_names.append(model_name)
        
    # Remove duplicates by converting the list to a set and then back to a list
    model_names = list(set(model_names))

    return model_names

########################################################################

# get faf-water contributing model names

directory_of_interest = '/oak/stanford/groups/earlew/zkaufman/Archive_KaufmanGRL2025/fafwater'
model_names_fafwater = extract_model_names(directory_of_interest,'faf-water')
print('faf-water contributing models:')
print(model_names_fafwater)


faf-water contributing models:
['ACCESS-CM2', 'HadGEM3-GC31-LL', 'CAS-ESM2-0', 'CanESM5', 'GFDL-ESM2M', 'MIROC6', 'CESM2', 'FGOALS-g3', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'MPI-ESM1-2-HR']


In [3]:
# initialize dask client for preprocessing. 

cluster = dask.distributed.LocalCluster\
(dashboard_address=':9000',n_workers=40,memory_limit='100GB')

client = dask.distributed.Client(cluster)
client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:9000/status,
Dashboard: http://127.0.0.1:9000/status,Workers: 40
Total threads: 40,Total memory: 3.64 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36311,Workers: 40
Dashboard: http://127.0.0.1:9000/status,Total threads: 40
Started: Just now,Total memory: 3.64 TiB
Comm: tcp://127.0.0.1:38796,Total threads: 1
Dashboard: http://127.0.0.1:45479/status,Memory: 93.13 GiB
Nanny: tcp://127.0.0.1:44851,


In [4]:
%%time 
%env PYTHONWARNINGS=ignore

# build dictionary for faf-water 

thetao_fafwater = {} 

########################################################################
# Homogenize attributes of the CMIP6 faf-water experiments, select uppermost level (if using thetao)
# take the annual mean. 

def process_dataset(model, experiment):
    os.chdir('/oak/stanford/groups/earlew/zkaufman/Archive_KaufmanGRL2025/fafwater')
    ds = xr.open_dataset(f'tos_{model}_{experiment}.nc', chunks={'time': 100})
    if model=='CAS-ESM2-0' or model=='FGOALS-g3':
        ds = broadcast_lonlat(rename_cmip6(ds)).thetao.sel(lev=0, method='nearest')
    else:
        ds = broadcast_lonlat(rename_cmip6(ds)).tos
    ds['time'] = xr.cftime_range(start='1000', periods=len(ds['time']), freq='ME') 
    ds = ds.resample(time='YE').mean()
    
    return ds
########################################################################

delayed_results = [dask.delayed(process_dataset)(model,'faf-water') for model in model_names_fafwater]
results = dask.compute(delayed_results)

thetao_fafwater = {f'thetao_{model}_faf-water': ds for model, ds in zip(model_names_fafwater, results[0])}

# fix land masking issue with FGOALS model 

thetao_fafwater['thetao_FGOALS-g3_faf-water'] = thetao_fafwater['thetao_FGOALS-g3_faf-water']\
.where(thetao_fafwater['thetao_FGOALS-g3_faf-water'] < 1e10, np.nan)



env: PYTHONWARNINGS=ignore


/home/groups/earlew/zkaufman/miniconda3/envs/zacks_gputorch/lib/python3.10/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'tos' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/home/groups/earlew/zkaufman/miniconda3/envs/zacks_gputorch/lib/python3.10/site-packages/xarray/coding/times.py:170: SerializationWarning: Ambiguous reference date string: 950-01-01. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0950-01-01). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/home/groups/earlew/zkaufman/miniconda3/envs/zacks_gputorch/lib/python3.10/site-packages/xarray/coding/times.py:987: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, re

CPU times: user 8.73 s, sys: 1.39 s, total: 10.1 s
Wall time: 51.8 s


In [5]:
%%time

# build second dictionary for regridded faf-water data. 

thetao_fafwater_rg = {}

########################################################################
# regrid to standard 1x1 grid using bilinear interpolation. 
# drop dimensions other than time,lat,lon

def regrid_dataarray(data):
    
    keep_coords = {'time', 'lat', 'lon'}
    drop_coords = set(data.coords) - keep_coords

    ds_out = xe.util.grid_global(1,1)
    ds_in = xr.DataArray.to_dataset(data.drop_vars(drop_coords))
    
    regridder = xe.Regridder(ds_in,ds_out, 'bilinear', periodic=True, ignore_degenerate=True)
    
    return regridder(data.drop_vars(drop_coords))
######################################################################## 

delayed_results = [dask.delayed(regrid_dataarray)(thetao_fafwater[key]) \
                   for key in thetao_fafwater.keys()]

results = dask.compute(delayed_results)

thetao_fafwater_rg = {f'thetao_{model}_faf-water': ds for model, ds in zip(model_names_fafwater, results[0])}

/home/groups/earlew/zkaufman/miniconda3/envs/zacks_gputorch/lib/python3.10/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


CPU times: user 10.5 s, sys: 1.69 s, total: 12.2 s
Wall time: 46.2 s


In [6]:
%%time 
%env PYTHONWARNINGS=ignore

# Access piControl data for these same models from the intake ESM datastore. 
# CAS-ESM2-0 and GFDL-ESM2M are not accessible here and are merged in later from saved output accessed on ESGF 

col = google_cmip_col()

df_base = col.search(
    activity_id=['CMIP'],
    table_id = ['Omon'],
    variable_id = ['thetao'],
    experiment_id = ['piControl']
)

# use faf-water attributes to find corresponding models and their ensemble members 

df_piControl = df_base.search(
    source_id = model_names_fafwater,
    member_id = ['r1i1p1f1','r1i1p2f1'] 
)

# set kwargs for preprocessing 

kwargs = {
    'zarr_kwargs':{
        'consolidated':True,
    },
    'aggregate':False,
    'preprocess':combined_preprocessing
}

# create xarray dictionary from search query. 

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    thetao_piControl_fw = df_piControl.to_dataset_dict(**kwargs)
    
    
# check for un-needed ensemble members and grid variants. 

for key in thetao_piControl_fw.keys():
    
    print(thetao_piControl_fw[key].attrs['source_id'], \
          thetao_piControl_fw[key].attrs['grid_label'],thetao_piControl_fw[key].attrs['variant_label'])

env: PYTHONWARNINGS=ignore

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


/home/groups/earlew/zkaufman/miniconda3/envs/zacks_gputorch/lib/python3.10/site-packages/xarray/coding/times.py:987: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/groups/earlew/zkaufman/miniconda3/envs/zacks_gputorch/lib/python3.10/site-packages/xarray/coding/times.py:987: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/groups/earlew/zkaufman/miniconda3/envs/zacks_gputorch/lib/python3.10/site-packages/xarray/coding/times.py:987: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _

MPI-ESM1-2-HR gn r1i1p1f1
MPI-ESM1-2-LR gn r1i1p1f1
CESM2 gn r1i1p1f1
CanESM5 gn r1i1p2f1
CanESM5 gn r1i1p1f1
MIROC6 gn r1i1p1f1
FGOALS-g3 gn r1i1p1f1
ACCESS-CM2 gn r1i1p1f1
CESM2 gr r1i1p1f1
HadGEM3-GC31-LL gn r1i1p1f1
MRI-ESM2-0 gr r1i1p1f1
CPU times: user 11.5 s, sys: 1.46 s, total: 12.9 s
Wall time: 59.4 s


In [7]:
# remove un-needed grid variants. 

thetao_piControl_fw = interpolate_grid_label(thetao_piControl_fw, target_grid_label='gn')

# find and remove CanESM5 ensemble member r1i1p1f1, since faf-water uses r1i1p1f2
# concat_members is an alternative option, but causes big dask delays. 

key_to_delete = None

for key, dataset in thetao_piControl_fw.items():
    if (dataset.attrs.get('source_id') == 'CanESM5' and
        dataset.attrs.get('variant_label') == 'r1i1p1f1'):
        
        key_to_delete = key
        break

if key_to_delete:
    
    del thetao_piControl_fw[key_to_delete]


# retitle intake_esm dictionary to be consistent with faf-water experiment dictionaries. 

keys_to_replace  = list(thetao_piControl_fw.keys())

for key in keys_to_replace:
    
    model = thetao_piControl_fw[key].attrs['intake_esm_attrs:source_id']

    thetao_piControl_fw[f'thetao_{model}_piControl'] = thetao_piControl_fw[key]
    
    del thetao_piControl_fw[key]

In [8]:
%%time

# pre-process and regrid piControl data as in faf-water, but with syntax optimized for intake_ESM data. 
# limit to first 100 years of piControl 

keys_intakesm = list(thetao_piControl_fw.keys())

# overwrite thetao_piControl_fw dictionary contents with annual mean of first 100 year for top vertical level 
# standardize time indexing 

for key in keys_intakesm:

    thetao_piControl_fw[key] = thetao_piControl_fw[key].thetao.squeeze()\
    .sel(lev=0,method='nearest').isel(time=slice(0,1200)).compute()
    
    thetao_piControl_fw[key]['time'] = \
    xr.cftime_range(start='1000', periods=len(thetao_piControl_fw[key]['time']), freq='M')    

# fix land masking issue with FGOALS model 

thetao_piControl_fw['thetao_FGOALS-g3_piControl'] = thetao_piControl_fw['thetao_FGOALS-g3_piControl']\
.where(thetao_piControl_fw['thetao_FGOALS-g3_piControl'] < 1e10, np.nan)


# then, put regridded data in additional dictionary

thetao_piControl_fw_rg = {}

for key in keys_intakesm:

    thetao_piControl_fw_rg[key] = regrid_dataarray(thetao_piControl_fw[key])

CPU times: user 4min 53s, sys: 44.9 s, total: 5min 38s
Wall time: 17min 37s


In [9]:
%%time 
%env PYTHONWARNINGS=ignore

# redefine process_dataset function for piControl data taken from ESGF 

########################################################################

def process_dataset(variable, model, experiment):
    os.chdir('/oak/stanford/groups/earlew/zkaufman/Archive_KaufmanGRL2025/fafwater/piControl_missingfromcloud')
    ds = xr.open_dataset(f'{variable}_{model}_{experiment}.nc', chunks={'time': 100})
    ds = broadcast_lonlat(rename_cmip6(ds)).tos
    ds['time'] = xr.cftime_range(start='1000', periods=len(ds['time']), freq='M') 
    ds = ds.resample(time='YE').mean()
    
    return ds

########################################################################

# retrieve two missing piControl models from ESGF 

thetao_piControl_fw_additions = {} 

delayed_results = [dask.delayed(process_dataset)('tos',model,'piControl') \
                   for model in ['CAS-ESM2-0','GFDL-ESM2M']]
results = dask.compute(delayed_results)

thetao_piControl_fw_additions = {f'thetao_{model}_piControl': ds \
                                 for model, ds in zip(['CAS-ESM2-0','GFDL-ESM2M'], results[0])}


# regrid two missing models from ESGF

thetao_piControl_fw_additions_rg = {} 

delayed_results = [dask.delayed(regrid_dataarray)(thetao_piControl_fw_additions[key]) \
                   for key in thetao_piControl_fw_additions.keys()]

results = dask.compute(delayed_results)

thetao_piControl_fw_additions_rg = {f'thetao_{model}_piControl': ds \
                                    for model, ds in zip(['CAS-ESM2-0','GFDL-ESM2M'], results[0])}



env: PYTHONWARNINGS=ignore


<timed exec>:11: FutureWarning: 'M' is deprecated and will be removed in a future version. Please use 'ME' instead of 'M'.
<timed exec>:11: FutureWarning: 'M' is deprecated and will be removed in a future version. Please use 'ME' instead of 'M'.
/home/groups/earlew/zkaufman/miniconda3/envs/zacks_gputorch/lib/python3.10/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


CPU times: user 12 s, sys: 2.23 s, total: 14.2 s
Wall time: 1min 3s


In [10]:
# merge ESGF and intake_esm sources

thetao_piControl_fw = {**thetao_piControl_fw,**thetao_piControl_fw_additions}
thetao_piControl_fw_rg = {**thetao_piControl_fw_rg,**thetao_piControl_fw_additions_rg}


In [11]:
# correct units for GFDL-ESM2M piControl  (deg. C to K)

thetao_piControl_fw_rg[f'thetao_GFDL-ESM2M_piControl'] = thetao_piControl_fw_rg[f'thetao_GFDL-ESM2M_piControl']-273
thetao_piControl_fw[f'thetao_GFDL-ESM2M_piControl'] = thetao_piControl_fw[f'thetao_GFDL-ESM2M_piControl']-273

In [13]:
%%time

# Initialize lists for SST anomalies

SST_anoms_fafwater = []

# Define a function to calculate anomalies for a single model 

def calculate_anomaly_fafwater(model):
    return (thetao_fafwater_rg[f'thetao_{model}_faf-water'][60:70,:,:].mean(dim='time')
            - thetao_piControl_fw_rg[f'thetao_{model}_piControl'].mean(dim='time'))

# Calculate anomalies for each experiment and append to the lists

SST_anoms_fafwater = [calculate_anomaly_fafwater(model) for model in model_names_fafwater]

# Convert lists to xarray DataArrays

SST_anoms_fafwater = \
xr.concat(SST_anoms_fafwater, dim='model', coords='minimal', compat='override')

SST_anoms_fafwater = SST_anoms_fafwater.assign_coords(model=model_names_fafwater)

CPU times: user 1.65 s, sys: 1.46 s, total: 3.11 s
Wall time: 2.64 s


In [14]:
# save postprocessed file for anomaly maps of each model. 

os.chdir('/oak/stanford/groups/earlew/zkaufman/Archive_KaufmanGRL2025/postprocessed_analysis_notebooks')

output_filename = 'SST_response_maps_fafwater.nc'
SST_anoms_fafwater.to_netcdf(output_filename)

In [16]:
# calculate spatially averaged SST response (unscaled, for climate response functions )

def subset_bylatitude(data, south_bound, north_bound):
    lat_mask = (data.lat >= south_bound) & (data.lat <= north_bound)
    data_SO = data.where(lat_mask, drop=True)
    return data_SO


# spatially average CMIP6 Omon field over specified latitudes
# assumes spatial average is conducted over all longitudes
# Assumes regrid_dataarray function is already applied
def spatial_average(inputdata,southlat,northlat):
    data_subset = subset_bylatitude\
    (inputdata,south_bound=southlat,north_bound=northlat)
    lat_subset = data_subset.lat
    coslat = np.cos(np.deg2rad(lat_subset))
    weight_factor = coslat / coslat.mean(dim='y')
    data_average = \
    (data_subset * weight_factor).mean(dim=('x', 'y'),skipna=True)
    return data_average

# Rename dimensions
thetao_piControl_fw['thetao_GFDL-ESM2M_piControl']\
= thetao_piControl_fw['thetao_GFDL-ESM2M_piControl'].rename({'rlon': 'x', 'rlat': 'y'})

# Initialize dictionary

fafwater_CRF_dict = {}

for model in model_names_fafwater:
    
    fafwater_CRF_dict[f'CRF_{model}_fafwater'] = \
    (spatial_average(thetao_fafwater[f'thetao_{model}_faf-water'].compute(),-65,-50) - \
    spatial_average(thetao_piControl_fw[f'thetao_{model}_piControl'].mean(dim='time').compute(),-65,-50)) 

In [17]:
# save postprocessed file for response functions. 

os.chdir('/oak/stanford/groups/earlew/zkaufman/Archive_KaufmanGRL2025/postprocessed_analysis_notebooks')

def save_data_arrays_to_netcdf(data_arrays, filename):
    first_data_array_saved = False

    for var_name, data_array in data_arrays.items():
        mode = 'w' if not first_data_array_saved else 'a'
        data_array.to_netcdf(filename, mode=mode, group=var_name)
        first_data_array_saved = True
        
        
save_data_arrays_to_netcdf(fafwater_CRF_dict, 'fafwater_CRFs_unscaled.nc')